# SQL Queries

We all have a list of queries that are useful that we keep. Maybe it would be good to have these for our team in a notebook

Maybe something for deadlocks

# Deadlocks

<a href="command:workbench.action.terminal.sendSequence?%7B%22text%22%3A%22 .%5c%5cdeadlockmaker.ps1 \u000D \u000D %22%7D">Lets create some deadlocks</a>

and then we can read them from the system_health 



In [11]:
DECLARE @system_health XML

-- Copy ring buffer into an XML variable
SELECT
	@system_health = CAST(xet.target_data AS XML)
FROM
	sys.dm_xe_session_targets xet
	INNER JOIN sys.dm_xe_sessions xe
		ON xe.address = xet.event_session_address
WHERE
	xe.name = 'system_health'
	AND xet.target_name = 'ring_buffer'

-- Get the time stamp and deadlock graph
SELECT
	xed.value('@timestamp','datetime'),
	xed.query('.')
FROM
	@system_health.nodes('RingBufferTarget/event[@name="xml_deadlock_report"]') AS XEventData(xed)

(5 rows affected)

Total execution time: 00:00:00.048

(No column name) (No column name) 2019-11-05 09:06:54.370 <event name="xml_deadlock_report" package="sqlserver" timestamp="2019-11-05T09:06:54.369Z"><data name="xml_report"><type name="xml" package="package0" /><value><deadlock><victim-list><victimProcess id="process102a39b468" /></victim-list><process-list><process id="process102a39b468" taskpriority="0" logused="0" waitresource="RID: 2:3:56:0" waittime="4687" ownerId="38946" transactionname="user_transaction" lasttranstarted="2019-11-05T09:06:49.680" XDES="0x103ad74428" lockMode="U" schedulerid="4" kpid="668" status="suspended" spid="60" sbid="0" ecid="0" priority="0" trancount="2" lastbatchstarted="2019-11-05T09:06:49.680" lastbatchcompleted="2019-11-05T09:06:49.680" lastattention="1900-01-01T00:00:00.680" clientapp="SQLCMD" hostname="BEARDXPS" hostpid="2880" loginname="sa" isolationlevel="read committed (2)" xactid="38946" currentdb="2" currentdbname="tempdb" lockTimeout="4294967295" clientoption1="536870944" clientoption2="128056"><executionStack><frame procname="adhoc" line="12" stmtstart="16" stmtend="82" sqlhandle="0x02000000d42f0219d0cbdad998dc30c178c61c84ce17200b0000000000000000000000000000000000000000">
unknown </frame><frame procname="adhoc" line="12" stmtstart="512" stmtend="570" sqlhandle="0x02000000e931a53346a3dfeddd0460cad8746111a05046120000000000000000000000000000000000000000">
unknown </frame></executionStack><inputbuf>
IF OBJECT_ID('tempdb..table1') IS NULL
BEGIN
	CREATE TABLE table1 (column1 int);
	INSERT INTO table1 VALUES (1);
END
IF OBJECT_ID('tempdb..table2') IS NULL
BEGIN
	CREATE TABLE table2 (column1 int);
	INSERT INTO table2 VALUES (1);
END
BEGIN TRAN
UPDATE table1
SET column1 = 0
DECLARE @waitString varchar(50) = 'WAITFOR DELAY ''00:00:'+ RIGHT('0' + CAST(ABS(CHECKSUM(NEWID())) % 10 AS varchar(2)),2) +''''
EXEC(@waitString)
UPDATE table2
SET column1 = 0
ROLLBACK
BEGIN TRAN
UPDATE table2
SET column1 = 0
SET @waitString = 'WAITFOR DELAY ''00:00:'+ RIGHT('0' + CAST(ABS(CHECKSUM(NEWID())) % 10 AS varchar(2)),2) +''''
EXEC(@waitString)
UPDATE table1
SET column1 = 0
ROLLBACK </inputbuf></process><process id="process1032c08108" taskpriority="0" logused="200" waitresource="RID: 2:1:368:0" waittime="2714" ownerId="38932" transactionname="user_transaction" lasttranstarted="2019-11-05T09:06:49.653" XDES="0x103ad54428" lockMode="U" schedulerid="4" kpid="512" status="suspended" spid="59" sbid="0" ecid="0" priority="0" trancount="2" lastbatchstarted="2019-11-05T09:06:49.650" lastbatchcompleted="2019-11-05T09:06:49.650" lastattention="1900-01-01T00:00:00.650" clientapp="SQLCMD" hostname="BEARDXPS" hostpid="5520" loginname="sa" isolationlevel="read committed (2)" xactid="38932" currentdb="2" currentdbname="tempdb" lockTimeout="4294967295" clientoption1="536870944" clientoption2="128056"><executionStack><frame procname="adhoc" line="16" stmtstart="16" stmtend="82" sqlhandle="0x0200000086a188362482cdd2ca19e40115cfd71fa325d2fc0000000000000000000000000000000000000000">
unknown </frame><frame procname="adhoc" line="16" stmtstart="874" stmtend="932" sqlhandle="0x02000000e931a53346a3dfeddd0460cad8746111a05046120000000000000000000000000000000000000000">
unknown </frame></executionStack><inputbuf>
IF OBJECT_ID('tempdb..table1') IS NULL
BEGIN
	CREATE TABLE table1 (column1 int);
	INSERT INTO table1 VALUES (1);
END
IF OBJECT_ID('tempdb..table2') IS NULL
BEGIN
	CREATE TABLE table2 (column1 int);
	INSERT INTO table2 VALUES (1);
END
BEGIN TRAN
UPDATE table1
SET column1 = 0
DECLARE @waitString varchar(50) = 'WAITFOR DELAY ''00:00:'+ RIGHT('0' + CAST(ABS(CHECKSUM(NEWID())) % 10 AS varchar(2)),2) +''''
EXEC(@waitString)
UPDATE table2
SET column1 = 0
ROLLBACK
BEGIN TRAN
UPDATE table2
SET column1 = 0
SET @waitString = 'WAITFOR DELAY ''00:00:'+ RIGHT('0' + CAST(ABS(CHECKSUM(NEWID())) % 10 AS varchar(2)),2) +''''
EXEC(@waitString)
UPDATE table1
SET column1 = 0
ROLLBACK </inputbuf></process><process id="process102bc2f468" taskpriori

Or maybe you want to do it with a seperate extended event session

In [21]:
CREATE EVENT SESSION [Deadlocks] ON SERVER
ADD EVENT sqlserver.xml_deadlock_report
ADD TARGET package0.ring_buffer(SET max_events_limit=(1000), max_memory=(2048))
WITH (MAX_MEMORY = 2048KB,
      EVENT_RETENTION_MODE = ALLOW_SINGLE_EVENT_LOSS,
      MAX_DISPATCH_LATENCY = 30 SECONDS,
      MAX_EVENT_SIZE = 0KB,
      MEMORY_PARTITION_MODE = NONE,
      TRACK_CAUSALITY = OFF,
      STARTUP_STATE = ON)
GO
-- Start the event session  
ALTER EVENT SESSION [Deadlocks] ON SERVER  
STATE = start;  
GO  

Commands completed successfully.

Commands completed successfully.

Total execution time: 00:00:00.059


<a href="command:workbench.action.terminal.sendSequence?%7B%22text%22%3A%22 .%5c%5cdeadlockmaker.ps1 \u000D \u000D %22%7D">Lets create some deadlocks</a>

In [23]:
SELECT
DATEADD(mi, DATEDIFF(mi, GETUTCDATE(), CURRENT_TIMESTAMP), DeadlockEventXML.value('(event/@timestamp)[1]', 'datetime2')) AS [EventTime],
DeadlockEventXML.value('(//process[@id[//victim-list/victimProcess[1]/@id]]/@hostname)[1]', 'nvarchar(max)') AS HostName,
DeadlockEventXML.value('(//process[@id[//victim-list/victimProcess[1]/@id]]/@clientapp)[1]', 'nvarchar(max)') AS ClientApp,
DB_NAME(DeadlockEventXML.value('(//process[@id[//victim-list/victimProcess[1]/@id]]/@currentdb)[1]', 'nvarchar(max)')) AS [DatabaseName],
DeadlockEventXML.value('(//process[@id[//victim-list/victimProcess[1]/@id]]/@transactionname)[1]', 'nvarchar(max)') AS VictimTransactionName,
DeadlockEventXML.value('(//process[@id[//victim-list/victimProcess[1]/@id]]/@isolationlevel)[1]', 'nvarchar(max)') AS IsolationLevel,
DeadlockEventXML.query('(event/data[@name="xml_report"]/value/deadlock)[1]') AS DeadLockGraph,
DeadlockEventXML
FROM
(
SELECT
XEvent.query('.') AS DeadlockEventXML,
Data.TargetData
FROM
(
SELECT
CAST(target_data AS XML) AS TargetData
FROM sys.dm_xe_session_targets st
JOIN sys.dm_xe_sessions s ON s.address = st.event_session_address
WHERE s.name = 'Deadlocks' AND
st.target_name = 'ring_buffer'
) AS Data
CROSS APPLY TargetData.nodes('RingBufferTarget/event[@name="xml_deadlock_report"]') AS XEventData(XEvent)
) AS DeadlockInfo 

(4 rows affected)

Total execution time: 00:00:00.141

EventTime HostName ClientApp DatabaseName VictimTransactionName IsolationLevel DeadLockGraph DeadlockEventXML 2019-11-05 09:15:09.0960000 BEARDXPS SQLCMD tempdb user_transaction read committed (2) <deadlock><victim-list><victimProcess id="process102a607088" /></victim-list><process-list><process id="process102a607088" taskpriority="0" logused="0" waitresource="RID: 2:3:56:0" waittime="14188" ownerId="67073" transactionname="user_transaction" lasttranstarted="2019-11-05T09:14:54.903" XDES="0x1034584428" lockMode="U" schedulerid="2" kpid="720" status="suspended" spid="71" sbid="0" ecid="0" priority="0" trancount="2" lastbatchstarted="2019-11-05T09:14:54.903" lastbatchcompleted="2019-11-05T09:14:54.903" lastattention="1900-01-01T00:00:00.903" clientapp="SQLCMD" hostname="BEARDXPS" hostpid="11616" loginname="sa" isolationlevel="read committed (2)" xactid="67073" currentdb="2" currentdbname="tempdb" lockTimeout="4294967295" clientoption1="536870944" clientoption2="128056"><executionStack><frame procname="adhoc" line="12" stmtstart="16" stmtend="82" sqlhandle="0x02000000d42f0219d0cbdad998dc30c178c61c84ce17200b0000000000000000000000000000000000000000">
unknown </frame><frame procname="adhoc" line="12" stmtstart="512" stmtend="570" sqlhandle="0x02000000e931a53346a3dfeddd0460cad8746111a05046120000000000000000000000000000000000000000">
unknown </frame></executionStack><inputbuf>
IF OBJECT_ID('tempdb..table1') IS NULL
BEGIN
	CREATE TABLE table1 (column1 int);
	INSERT INTO table1 VALUES (1);
END
IF OBJECT_ID('tempdb..table2') IS NULL
BEGIN
	CREATE TABLE table2 (column1 int);
	INSERT INTO table2 VALUES (1);
END
BEGIN TRAN
UPDATE table1
SET column1 = 0
DECLARE @waitString varchar(50) = 'WAITFOR DELAY ''00:00:'+ RIGHT('0' + CAST(ABS(CHECKSUM(NEWID())) % 10 AS varchar(2)),2) +''''
EXEC(@waitString)
UPDATE table2
SET column1 = 0
ROLLBACK
BEGIN TRAN
UPDATE table2
SET column1 = 0
SET @waitString = 'WAITFOR DELAY ''00:00:'+ RIGHT('0' + CAST(ABS(CHECKSUM(NEWID())) % 10 AS varchar(2)),2) +''''
EXEC(@waitString)
UPDATE table1
SET column1 = 0
ROLLBACK </inputbuf></process><process id="process102a39a108" taskpriority="0" logused="200" waitresource="RID: 2:1:368:0" waittime="7200" ownerId="67059" transactionname="user_transaction" lasttranstarted="2019-11-05T09:14:54.893" XDES="0x103ad74428" lockMode="U" schedulerid="4" kpid="600" status="suspended" spid="70" sbid="0" ecid="0" priority="0" trancount="2" lastbatchstarted="2019-11-05T09:14:54.893" lastbatchcompleted="2019-11-05T09:14:54.893" lastattention="1900-01-01T00:00:00.893" clientapp="SQLCMD" hostname="BEARDXPS" hostpid="2332" loginname="sa" isolationlevel="read committed (2)" xactid="67059" currentdb="2" currentdbname="tempdb" lockTimeout="4294967295" clientoption1="536870944" clientoption2="128056"><executionStack><frame procname="adhoc" line="16" stmtstart="16" stmtend="82" sqlhandle="0x0200000086a188362482cdd2ca19e40115cfd71fa325d2fc0000000000000000000000000000000000000000">
unknown </frame><frame procname="adhoc" line="16" stmtstart="874" stmtend="932" sqlhandle="0x02000000e931a53346a3dfeddd0460cad8746111a05046120000000000000000000000000000000000000000">
unknown </frame></executionStack><inputbuf>
IF OBJECT_ID('tempdb..table1') IS NULL
BEGIN
	CREATE TABLE table1 (column1 int);
	INSERT INTO table1 VALUES (1);
END
IF OBJECT_ID('tempdb..table2') IS NULL
BEGIN
	CREATE TABLE table2 (column1 int);
	INSERT INTO table2 VALUES (1);
END
BEGIN TRAN
UPDATE table1
SET column1 = 0
DECLARE @waitString varchar(50) = 'WAITFOR DELAY ''00:00:'+ RIGHT('0' + CAST(ABS(CHECKSUM(NEWID())) % 10 AS varchar(2)),2) +''''
EXEC(@waitString)
UPDATE table2
SET column1 = 0
ROLLBACK
BEGIN TRAN
UPDATE table2
SET column1 = 0
SET @waitString = 'WAITFOR DELAY ''00:00:'+ RIGHT('0' + CAST(ABS(CHECKSUM(NEWID())) % 10 AS varchar(2)),2) +''''
EXEC(@waitString)
UPDATE table1
SET column1 = 0
ROLLBACK </inputbuf></process><process id="process102f03f468" taskpriority="0" logused="200